### Install Postgres on Raspberry Pi

Follow the instructions in this article: [Setting up a PostgreSQL Database on a Raspberry Pi](https://pimylifeup.com/raspberry-pi-postgresql/)


In [1]:
%%bash
sudo apt update -y
sudo apt install -y postgresql

Hit:1 https://download.docker.com/linux/raspbian buster InRelease
Hit:2 http://archive.raspberrypi.org/debian buster InRelease
Hit:3 http://raspbian.raspberrypi.org/raspbian buster InRelease
Reading package lists...
Building dependency tree...
Reading state information...
98 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
postgresql is already the newest version (11+200+deb10u4).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [2]:
%%html
<img src="secure-postgres-8x.gif" width="1200px" />

### Install nginx

In [1]:
%%bash
sudo apt install -y nginx
sudo service nginx status

● nginx.service - A high performance web server and a reverse proxy server
   Loaded: loaded (/lib/systemd/system/nginx.service; enabled; vendor preset: enabled)
   Active: active (running) since Sun 2022-05-29 00:19:26 PDT; 1 day 14h ago
     Docs: man:nginx(8)
 Main PID: 562 (nginx)
    Tasks: 5 (limit: 4915)
   CGroup: /system.slice/nginx.service
           ├─ 562 nginx: master process /usr/sbin/nginx -g daemon on; master_process on;
           ├─1206 nginx: worker process
           ├─1207 nginx: worker process
           ├─1208 nginx: worker process
           └─1209 nginx: worker process

May 29 00:19:25 raspberrypi systemd[1]: Starting A high performance web server and a reverse proxy server...
May 29 00:19:26 raspberrypi systemd[1]: Started A high performance web server and a reverse proxy server.


### Enable SSL

Open ports 80 and 443 on your router, and forward them to Raspberry Pi

<img src="eero-raspberry-pi.png" width="400px">
<img src="eero-web-80.png" width="400px">
<img src="eero-web-443.png" width="400px">

#### Map a domain name to your external IP address using Route 53

In [3]:
# Find out your external IP address
!curl ipconfig.io/ip

71.202.157.221


<img src="route53-create-record.png" width="600px">

#### Confirm that you are able to reach your web server using the domain name

If XFinity is your Internet service provider, this may only work from OUTSIDE the local network. Switch to your mobile Internet connection before proceeding

eg. http://pi400.cloudmatica.com


#### Upgrade your connection to https by running certbot

In [4]:
!sudo apt install -y certbot python-certbot-nginx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
certbot is already the newest version (0.31.0-1+deb10u1).
python-certbot-nginx is already the newest version (0.31.0-1).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [ ]:
!sudo certbot --nginx --domains pi400.cloudmatica.com

<img src="certbot.gif" width="800px">

### Set postgres to use your certbot/nginx certificate

#### First find the location of your postgresql.conf

In [5]:
!sudo find / -name 'postgresql.conf'

/usr/lib/tmpfiles.d/postgresql.conf
/etc/postgresql/11/main/postgresql.conf
find: ‘/run/user/1000/gvfs’: Permission denied


In [7]:
!head -n 50 /etc/postgresql/11/main/postgresql.conf

# -----------------------------
# PostgreSQL configuration file
# -----------------------------
#
# This file consists of lines of the form:
#
#   name = value
#
# (The "=" is optional.)  Whitespace may be used.  Comments are introduced with
# "#" anywhere on a line.  The complete list of parameter names and allowed
# values can be found in the PostgreSQL documentation.
#
# The commented-out settings shown in this file represent the default values.
# Re-commenting a setting is NOT sufficient to revert it to the default value;
# you need to reload the server.
#
# This file is read on server startup and when the server receives a SIGHUP
# signal.  If you edit the file on a running system, you have to SIGHUP the
# server for the changes to take effect, run "pg_ctl reload", or execute
# "SELECT pg_reload_conf()".  Some parameters, which are marked below,
# require a server shutdown and restart to take effect.
#
# Any parameter can also be given as a command-line option to the server, e.g.,

#### Note the data_directory and hba_file locations above

Also note that postgres uses "snakeoil" certificates by default. You can find them defined in postgresql.conf under `ssl_cert_file` and `ssl_key_file`

#### Copy the cert and key file from Letsencrypt into the Posgres data_directory

[Secure TCP/IP Connections with SSL](https://www.postgresql.org/docs/9.1/ssl-tcp.html)
Note that $PGDATA corresponds to the data_directory above

In [11]:
%%bash
export PGDATA=/var/lib/postgresql/11/main
# Copy the cert file
sudo cp /etc/letsencrypt/live/pi400.cloudmatica.com/fullchain.pem $PGDATA/server.crt
sudo cp /etc/letsencrypt/live/pi400.cloudmatica.com/privkey.pem $PGDATA/server.key
sudo chmod 644 $PGDATA/server.crt
sudo chmod 600 $PGDATA/server.key
sudo chown postgres:postgres $PGDATA/server.crt
sudo chown postgres:postgres $PGDATA/server.key

#### Update postgresql.conf with the new ssl_cert_file and ssl_key_file

#### Also, set the listen_address to * (all IP addresses)

If you do a colordiff from the original, you should see something like the following

<img src="colordiff-postgresql.conf.png" width="800px">

In [10]:
%%bash
# Restart postgresql service for the changes to take effect
sudo service postgresql restart
sudo service postgresql status

● postgresql.service - PostgreSQL RDBMS
   Loaded: loaded (/lib/systemd/system/postgresql.service; enabled; vendor preset: enabled)
   Active: active (exited) since Mon 2022-05-30 15:52:09 PDT; 43ms ago
  Process: 7460 ExecStart=/bin/true (code=exited, status=0/SUCCESS)
 Main PID: 7460 (code=exited, status=0/SUCCESS)

May 30 15:52:09 raspberrypi systemd[1]: Starting PostgreSQL RDBMS...
May 30 15:52:09 raspberrypi systemd[1]: Started PostgreSQL RDBMS.


#### Forward port 5432 to your postgres server

<img src="eero-postgres-5432.png" width="400px">

#### Modify pg_hba.conf to allow connections to pi database

Change to /etc/postgresql/11/main/pg_hba.conf should look like the following:

<img src="colordiff-pg_hba.conf.png" width="800px">

In [1]:
!sudo service postgresql restart